This is where I'm importing the gauge data from the 2 USGS tab-separated files I've got and the waterwatch website, as well as organizing it into a useful form. This is all going to be very-specifcally set up for my specific instance just because I really don't forsee using it for anything else

In [1]:
import urllib3 as url
import re
import datetime as dt
import pickle
http = url.PoolManager()

In [99]:
#This first one pulls each gauge number and it's lat lon coords
gauge_filepath = r'C:\Users\bpara\Geo data\Site coordinates.txt'
raw_coords = open(gauge_filepath, 'r')
gauge_final = dict()
first_line = True
for x in raw_coords:
    if first_line: 
        first_line = False
        continue
    if not x.split()[0] in gauge_final:
        temp = {"lat":x.split("	")[7], "lon":x.split("	")[8]}
        gauge_final[x.split()[0]] = temp

In [2]:
with open("USGS Gauge data", 'rb') as f:
        gauge_final = pickle.load(f)

In [ ]:
#And this one gets the time series for flooding and the flood stages (if avaliable) for them
for gauge in gauge_final.keys():
    t1 = dt.datetime.now()
    if 'hgt' in gauge_final[gauge]: continue
    site_num = gauge
    site = http.request('GET', "https://nwis.waterdata.usgs.gov/usa/nwis/uv/?referred_module=sw&search_site_no="+site_num+"&search_site_no_match_type=exact&index_pmcode_00065=1&group_key=NONE&sitefile_output_format=html_table&column_name=agency_cd&column_name=site_no&column_name=station_nm&range_selection=date_range&begin_date=2019-01-01&end_date=2019-12-31&format=rdb&date_format=YYYY-MM-DD&rdb_compression=value&list_of_search_criteria=search_site_no%2Crealtime_parameter_selection")
    time_list = []
    val_list = []
    list_1 = re.split("[\n]", site.data.decode("utf-8"))
    for line in list_1:
        if line[0:4] == "USGS":
            list_2 = line.split("\t")
            val_list.append(list_2[4])
            time_list.append(dt.datetime.strptime(list_2[2], '%Y-%m-%d %H:%M'))
    gauge_final[gauge]["time"] = time_list.copy()
    gauge_final[gauge]["hgt"] = val_list.copy()
    site2 = http.request('GET', "https://waterwatch.usgs.gov/webservices/floodstage?format=json&site=" + site_num)
    gauge_final[gauge]["flood"] = site2.data.decode("utf-8")
    file = open("USGS Gauge data", "wb")
    pickle.dump(gauge_final, file)
    file.close()
    print("Done in " + str(dt.datetime.now() - t1))

Done in 0:02:02.872666
Done in 0:01:49.664138
